<a href="https://colab.research.google.com/github/yingzibu/MOL2ADMET/blob/main/examples/GRU_VAE_JAK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -R 'a_inhibitor_design'
! git clone https://github.com/yingzibu/a_inhibitor_design.git --quiet
!rm -R 'JAK_ML'
! git clone https://github.com/yingzibu/JAK_ML.git --quiet
!rm -R 'JAK-MTATFP'
! git clone https://github.com/Yimeng-Wang/JAK-MTATFP.git --quiet

rm: cannot remove 'a_inhibitor_design': No such file or directory
rm: cannot remove 'JAK_ML': No such file or directory
rm: cannot remove 'JAK-MTATFP': No such file or directory


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install PyTDC --quiet
! pip install selfies  --quiet
! pip install pubchempy --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from tdc.generation import MolGen
import selfies as sf
from tqdm import tqdm
from torch.nn.utils.rnn import pack_sequence
from torch.utils.data import DataLoader

data = MolGen(name = 'ZINC')
split = data.get_split()
train = split['train']
valid = split['valid']
test = split['test']
# len(test)

from tdc import Oracle
qed = Oracle(name='QED')
sa = Oracle(name='SA')
qed('CCC'), sa('CCC')

Found local copy...
Loading...
Done!
100%|██████████| 9.05M/9.05M [00:00<00:00, 10.3MiB/s]
Done!


(0.3854706587740357, 1.7549570386349824)

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np

train_path = '/content/JAK-MTATFP/Data/JAK_train.csv'
valid_path = '/content/JAK-MTATFP/Data/JAK_valid.csv'
test_path = '/content/JAK-MTATFP/Data/JAK_test.csv'
train_df = pd.read_csv(train_path)
valid_df = pd.read_csv(valid_path)
test_df = pd.read_csv(test_path)
train_df.head(), train_df.shape, valid_df.shape, test_df.shape

(                                              SMILES  JAK1  pIC50_JAK1 JAK2  \
 0  [2H]C([2H])([2H])N=C(O)c1cnc(N=C(O)C2CC2)cc1Nc...   NaN         NaN  NaN   
 1  [2H]C([2H])([2H])NC(=O)c1cnc(NC(=O)C2CC2)cc1Nc...   NaN         NaN  NaN   
 2  [2H]C([2H])([2H])NC(=O)c1n[nH]c(=NC(=O)C2CC2)c...   NaN         NaN  NaN   
 3  [2H]C([2H])([2H])NC(=O)c1n[nH]c(=Nc2ccc(F)cn2)...   NaN         NaN  NaN   
 4  [2H]C([2H])([2H])NC(=O)c1nnc(NC(=O)C2CC2)cc1Nc...   1.0         0.0  NaN   
 
    pIC50_JAK2 JAK3  pIC50_JAK3   TYK2  pIC50_TYK2  
 0         NaN  NaN         NaN     13   -1.113943  
 1         NaN  NaN         NaN     13   -1.113943  
 2         NaN  NaN         NaN  50000   -4.698970  
 3         NaN  NaN         NaN    200   -2.301030  
 4         NaN  NaN         NaN    NaN         NaN  ,
 (10823, 9),
 (1529, 9),
 (1544, 9))

In [ ]:
enzymes = ['JAK1', 'JAK2', 'JAK3', 'TYK2']
col_names = ['pIC50_'+enzyme for enzyme in enzymes]

train_df_ = train_df[['SMILES'] + col_names].reset_index(drop=True)
# train_df_

train_ = train.reindex(columns = train.columns.tolist() + col_names)
# train_.rename(columns={'smiles': 'SMILES'})
train_ = train_.rename(columns={'smiles': 'SMILES'})

In [ ]:
for smi in tqdm(train_df_['SMILES'].tolist(),
                total=len(train_df_['SMILES'].tolist())
                desc='check duplicates'):
    if smi in train_['SMILES'].tolist():
        print(smi)

100%|██████████| 10823/10823 [00:56<00:00, 190.87it/s]


In [ ]:
train = pd.merge(train_df_, train_, how='outer').reset_index(drop=True)
train

,SMILES,pIC50_JAK1,pIC50_JAK2,pIC50_JAK3,pIC50_TYK2
0,[2H]C([2H])([2H])N=C(O)c1cnc(N=C(O)C2CC2)cc1Nc...,NaN,NaN,NaN,-1.113943
1,[2H]C([2H])([2H])NC(=O)c1cnc(NC(=O)C2CC2)cc1Nc...,NaN,NaN,NaN,-1.113943
2,[2H]C([2H])([2H])NC(=O)c1n[nH]c(=NC(=O)C2CC2)c...,NaN,NaN,NaN,-4.698970
3,[2H]C([2H])([2H])NC(=O)c1n[nH]c(=Nc2ccc(F)cn2)...,NaN,NaN,NaN,-2.301030
4,[2H]C([2H])([2H])NC(=O)c1nnc(NC(=O)C2CC2)cc1Nc...,0.0,NaN,NaN,NaN
...,...,...,...,...,...
185436,COc1cc(C[NH2+]C[C@@H](C)C[C@@H](C)O)cc(Br)c1O,NaN,NaN,NaN,NaN
185437,COC(=O)c1cccc2nc(C=O)[nH]c12,NaN,NaN,NaN,NaN
185438,CC1(C)CC[C@H](CNC(=O)Cn2ncc3ccccc3c2=O)c2ccccc21,NaN,NaN,NaN,NaN
185439,Cn1ccnc1C(=O)c1ccc(NC(=O)C2CCN(C(=O)C(C)(C)C)C...,NaN,NaN,NaN,NaN


In [ ]:
from torch.nn.utils.rnn import pack_sequence, pad_sequence
# from torch.utils.data import DataLoader
from torch.utils.data import DataLoader, Dataset
import torch
chars = set()

enzymes = ['JAK1', 'JAK2', 'JAK3', 'TYK2']
property_names = ['pIC50_'+enzyme for enzyme in enzymes]
print(property_names)

for string in train['SMILES']:
    chars.update(string)
all_sys =  ['<pad>', '<bos>', '<eos>', '<unk>'] + sorted(list(chars))
vocab = all_sys
c2i = {c: i for i, c in enumerate(all_sys)}
i2c = {i: c for i, c in enumerate(all_sys)}
print('len alphabet: ', len(c2i))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vector = torch.eye(len(c2i))

def char2id(char):
    if char not in c2i:
        return c2i['<unk>']
    else:
        return c2i[char]


def id2char(id):
    if id not in i2c:
        return i2c[c2i['<unk>']]
    else:
        return i2c[id]

def string2ids(string,add_bos=False, add_eos=False):
    ids = [char2id(c) for c in string]
    if add_bos:
        ids = [c2i['<bos>']] + ids
    if add_eos:
        ids = ids + [c2i['<eos>']]
    return ids
def ids2string(ids, rem_bos=True, rem_eos=True):
    # print(ids)
    if isinstance(ids[0], list): ids = ids[0]
    if len(ids) == 0:
        return ''
    if rem_bos and ids[0] == c2i['<bos>']:
        ids = ids[1:]
    # delete <eos>
    if rem_eos:
        for i, id in enumerate(ids):
            # print(i, id)
            if id == c2i['<eos>']:
                ids = ids[:i]
                break

    # if rem_eos and ids[-1] == c2i['<eos>']:
    #     ids = ids[:-1]

    string = ''.join([id2char(id) for id in ids])
    return string

def string2tensor(string, device='model'):
    ids = string2ids(string, add_bos=True, add_eos=True)
    tensor = torch.tensor(ids, dtype=torch.long,device=device if device == 'model' else device)
    return tensor

# tensor = [string2tensor(string, device=device) for string in train['smiles']]


vector = torch.eye(len(c2i))  # (alphabet_len, alphabet_len)
class jak_dataset(Dataset):
    def __init__(self, df, property_names, device, mask):
        super(jak_dataset, self).__init__()
        df = df.fillna(mask)
        df = df.reset_index(drop=True)
        self.df = df
        self.device = device
        self.len = len(df)
        self.col_names = property_names

    def __getitem__(self, idx):
        label = [self.df[col_name][idx] for col_name in self.col_names]
        label = torch.tensor(label, dtype=torch.float32).to(self.device)
        smi = self.df['SMILES'][idx]
        tensor = string2tensor(smi, device=self.device)
        return [tensor, label]
    def __len__(self): return self.len

def my_collate(batch):
    # batch contains a list of tuples of structure (sequence, target)
    data = [item[0] for item in batch]
    data = pad_sequence(data, batch_first=True, padding_value=c2i['<pad>'])
    # targets = [item[1] for item in batch]
    targets = [item[1] for item in batch]
    targets = torch.stack(targets)
    return (data, targets)

['pIC50_JAK1', 'pIC50_JAK2', 'pIC50_JAK3', 'pIC50_TYK2']
len alphabet:  40


In [ ]:
# -*- coding: utf-8 -*-
import torch
import torch.nn as nn
import torch.nn.functional as F

device = 'cuda'
MASK = -100

q_bidir = True
q_d_h = 256
q_n_layers = 1
q_dropout = 0.5
d_n_layers = 3
d_dropout = 0
d_z = 128 * 2 ############################################
d_d_h = 512
# from data import *

###########for classifier#################
h_dims = [128, 64]
out_dim = 4 ##############################
dims_classifier = [d_z, h_dims, out_dim]
pred_loss_percent = 0.5
MASK = -100 # if there is no label info
##########################################


from torch.optim.lr_scheduler import _LRScheduler
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn.utils import clip_grad_norm_
import math
import numpy as np
from collections import UserList, defaultdict
n_last = 1000
n_batch = 512
kl_start = 0
kl_w_start = 0.0
kl_w_end = 1.0 * 0.5 ###############################
n_epoch = 500
n_workers = 0

clip_grad  = 50
lr_start = 0.001
lr_n_period = 50
lr_n_mult = 1
lr_end = 3 * 1e-4
lr_n_restarts = 1 ###############################

In [ ]:
loader_jak = DataLoader(jak_dataset(train, property_names, device=device,
                                    mask=MASK),batch_size=n_batch, shuffle=True,
                        num_workers=n_workers, collate_fn=my_collate)

In [ ]:
# -*- coding: utf-8 -*-
import torch
import torch.nn as nn
import torch.nn.functional as F

class Regressor(nn.Module):
    """
    simple regression, for prediction using
    latent space of dim [batch_size, z_dim]
    """
    def __init__(self, dims):
        """
        :param dims [in_dim, h_dims, out_dim]
            in_dim: input dim
            h_dims: a list of hidden dims
            out_dim: output dim
        """
        super(Regressor, self).__init__()
        [in_dim, h_dims, out_dim] = dims
        self.dims = dims

        neurons = [in_dim, *h_dims]
        linear_layers = [nn.Linear(
            neurons[i-1], neurons[i]
            ) for i in range(1, len(neurons))]

        self.hidden = nn.ModuleList(linear_layers)
        self.final = nn.Linear(h_dims[-1], out_dim)

    def forward(self, x, y, loss_func=nn.MSELoss()):
        # print('classifier in x dim:', x.shape) [bs, h_dim]
        for layer in self.hidden:
            x = F.relu(layer(x))
            # print(x.shape) [batch_size, h_dim[-1]]
        x = self.final(x)
        mask = y == MASK
        # print(x.shape) # [bs, y_dim]
        # loss_func = nn.MSELoss()
        classify_loss  = loss_func(x[~mask], y[~mask])
        return x, classify_loss

    # def loss_function(self): return nn.MSELoss()

class VAE(nn.Module):
    def __init__(self,vocab,vector):
        super().__init__()
        self.vocabulary = vocab
        self.vector = vector

        n_vocab, d_emb = len(vocab), vector.size(1)
        self.x_emb = nn.Embedding(n_vocab, d_emb, c2i['<pad>'])
        self.x_emb.weight.data.copy_(vector)

        #ENCODER

        self.encoder_rnn = nn.GRU(d_emb,q_d_h,num_layers=q_n_layers,
                batch_first=True,dropout=q_dropout if q_n_layers > 1 else 0,
                bidirectional=q_bidir)
        q_d_last = q_d_h * (2 if q_bidir else 1)
        self.q_mu = nn.Linear(q_d_last, d_z)
        self.q_logvar = nn.Linear(q_d_last, d_z)



        # Decoder
        self.decoder_rnn = nn.GRU(d_emb + d_z,d_d_h,num_layers=d_n_layers,batch_first=True,dropout=d_dropout if d_n_layers > 1 else 0)
        self.decoder_latent = nn.Linear(d_z, d_d_h)
        self.decoder_fullyc = nn.Linear(d_d_h, n_vocab)

        # Classifier
        self.classifier = Regressor(dims_classifier)

        # Grouping the model's parameters
        self.encoder = nn.ModuleList([self.encoder_rnn,self.q_mu,self.q_logvar])
        self.decoder = nn.ModuleList([self.decoder_rnn,self.decoder_latent,self.decoder_fullyc])
        self.vae = nn.ModuleList([self.x_emb,self.encoder,self.decoder, self.classifier])



    @property
    def device(self):
        return next(self.parameters()).device

    def string2tensor(self, string, device='model'):
        ids = string2ids(string, add_bos=True, add_eos=True)
        tensor = torch.tensor(ids, dtype=torch.long,
                              device=self.device if device == 'model' else device)
        return tensor

    def tensor2string(self, tensor):
        ids = tensor.tolist()
        string = ids2string(ids, rem_bos=True, rem_eos=True)
        return string

    def forward(self,x, y=None):
        z, mu, kl_loss = self.forward_encoder(x)
        if y != None: y, classify_loss = self.classifier(mu, y)
        else: classify_loss = torch.tensor([0]).to(self.device)
        _, recon_loss = self.forward_decoder(x, z)
        # print("forward")
        return kl_loss, recon_loss, classify_loss

    def reconstruct(self, x):
        z, _, _ = self.forward_encoder(x)
        x, _ = self.forward_decoder(x, z)
        del z
        return x

    def reconstruct_mu(self, x):
        _, mu, _ = self.forward_encoder(x)
        x, _ = self.forward_decoder(x, mu)
        del mu
        return x

    def reparam(self, mu, logvar):
        eps = torch.randn_like(mu)
        z = mu + (logvar / 2).exp() * eps
        kl_loss = 0.5 * (logvar.exp() + mu ** 2 - 1 - logvar).sum(1).mean()
        return z,  kl_loss


    def forward_encoder(self, x):
        x = [self.x_emb(i_x) for i_x in x]
        x = nn.utils.rnn.pack_sequence(x)
        _, h = self.encoder_rnn(x, None)
        h = h[-(1 + int(self.encoder_rnn.bidirectional)):]
        h = torch.cat(h.split(1), dim=-1).squeeze(0)
        mu, logvar = self.q_mu(h), self.q_logvar(h)
        z, kl_loss = self.reparam(mu, logvar)
        # eps = torch.randn_like(mu)
        # z = mu + (logvar / 2).exp() * eps
        # kl_loss = 0.5 * (logvar.exp() + mu ** 2 - 1 - logvar).sum(1).mean()
        del x
        del h
        del logvar

        return z, mu, kl_loss

    def forward_decoder(self,x, z):
        lengths = [len(i_x) for i_x in x]
        x = nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value= c2i['<pad>'])
        x_emb = self.x_emb(x)
        z_0 = z.unsqueeze(1).repeat(1, x_emb.size(1), 1)
        x_input = torch.cat([x_emb, z_0], dim=-1)
        x_input = nn.utils.rnn.pack_padded_sequence(x_input, lengths, batch_first=True)
        h_0 = self.decoder_latent(z)
        h_0 = h_0.unsqueeze(0).repeat(self.decoder_rnn.num_layers, 1, 1)
        output, _ = self.decoder_rnn(x_input, h_0)
        output, _ = nn.utils.rnn.pad_packed_sequence(output, batch_first=True)
        y = self.decoder_fullyc(output)

        recon_loss = F.cross_entropy(y[:, :-1].contiguous().view(-1, y.size(-1)),
                                    x[:, 1:].contiguous().view(-1),
                                     ignore_index= c2i['<pad>'])
        return y, recon_loss


    def sample_z_prior(self,n_batch):
        return torch.randn(n_batch,self.q_mu.out_features,
                           device= self.x_emb.weight.device)

    def sample(self,n_batch, max_len=100, z=None, temp=1.0):
        with torch.no_grad():
            if z is None: z = self.sample_z_prior(n_batch)

            z = z.to(self.device)
            z_0 = z.unsqueeze(1)
            h = self.decoder_latent(z)
            h = h.unsqueeze(0).repeat(self.decoder_rnn.num_layers, 1, 1)
            w = torch.tensor(c2i['<bos>'], device=self.device).repeat(n_batch)
            x = torch.tensor([c2i['<pad>']], device=device).repeat(n_batch, max_len)
            x[:, 0] = c2i['<bos>']
            end_pads = torch.tensor([max_len], device=self.device).repeat(n_batch)
            eos_mask = torch.zeros(n_batch, dtype=torch.uint8, device=self.device)

            for i in range(1, max_len):
                x_emb = self.x_emb(w).unsqueeze(1)
                x_input = torch.cat([x_emb, z_0], dim=-1)

                o, h = self.decoder_rnn(x_input, h)
                y = self.decoder_fullyc(o.squeeze(1))
                y = F.softmax(y / temp, dim=-1)

                w = torch.multinomial(y, 1)[:, 0]
                x[~eos_mask, i] = w[~eos_mask]
                i_eos_mask = ~eos_mask & (w == c2i['<eos>'])
                end_pads[i_eos_mask] = i + 1
                eos_mask = eos_mask | i_eos_mask

                new_x = []
                for i in range(x.size(0)):
                    new_x.append(x[i, :end_pads[i]])

        return [self.tensor2string(i_x) for i_x in new_x]


In [ ]:
def get_collate_device(model):
    return model.device

def get_optim_params(model):
    return (p for p in model.parameters() if p.requires_grad)


def _n_epoch():
    return sum(lr_n_period * (lr_n_mult ** i) for i in range(lr_n_restarts))

class KLAnnealer:
    def __init__(self,n_epoch):
        self.i_start = kl_start
        self.w_start = kl_w_start
        self.w_max = kl_w_end
        self.n_epoch = n_epoch

        self.inc = (self.w_max - self.w_start) / (self.n_epoch - self.i_start)

    def __call__(self, i):
        k = (i - self.i_start) if i >= self.i_start else 0
        return self.w_start + k * self.inc


class CosineAnnealingLRWithRestart(_LRScheduler):
    def __init__(self , optimizer):
        self.n_period = lr_n_period
        self.n_mult = lr_n_mult
        self.lr_end = lr_end

        self.current_epoch = 0
        self.t_end = self.n_period

        # Also calls first epoch
        super().__init__(optimizer, -1)

    def get_lr(self):
        return [self.lr_end + (base_lr - self.lr_end) *
                (1 + math.cos(math.pi * self.current_epoch / self.t_end)) / 2
                for base_lr in self.base_lrs]

    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
        self.last_epoch = epoch
        self.current_epoch += 1

        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

        if self.current_epoch == self.t_end:
            self.current_epoch = 0
            self.t_end = self.n_mult * self.t_end


class CircularBuffer:
    def __init__(self, size):
        self.max_size = size
        self.data = np.zeros(self.max_size)
        self.size = 0
        self.pointer = -1

    def add(self, element):
        self.size = min(self.size + 1, self.max_size)
        self.pointer = (self.pointer + 1) % self.max_size
        self.data[self.pointer] = element
        return element

    def last(self):
        assert self.pointer != -1, "Can't get an element from an empty buffer!"
        return self.data[self.pointer]

    def mean(self):
        return self.data.mean()


class Logger(UserList):
    def __init__(self, data=None):
        super().__init__()
        self.sdata = defaultdict(list)
        for step in (data or []):
            self.append(step)

    def __getitem__(self, key):
        if isinstance(key, int):
            return self.data[key]
        elif isinstance(key, slice):
            return Logger(self.data[key])
        else:
            ldata = self.sdata[key]
            if isinstance(ldata[0], dict):
                return Logger(ldata)
            else:
                return ldata

    def append(self, step_dict):
        super().append(step_dict)
        for k, v in step_dict.items():
            self.sdata[k].append(v)


In [ ]:
# from tqdm import tqdm
import pandas as pd
n_samples = 3000
n_jobs = 1
max_len = 100

class sample():
    def take_samples(model,n_batch, max_len=max_len, n_samples=n_samples):
        n = n_samples
        samples = []
        with tqdm(total=n_samples, desc='Generating samples') as T:
            while n > 0:
                current_samples = model.sample(min(n, n_batch), max_len)
                samples.extend(current_samples)
                n -= len(current_samples)
                T.update(len(current_samples))
        samples = pd.DataFrame(samples, columns=['SMILES'])
        return samples

from IPython.display import display
from rdkit import Chem
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

m = Chem.MolFromSmiles

def visual_mols(sample_smis, return_valid_rate=True):

    valid_smis = []
    for i in sample_smis.SMILES:
        try:
            mol = m(i)
            if mol != None: valid_smis.append(i)
        except: pass
    valid_rate = len(valid_smis) / len(sample_smis)
    unique_smis = [*set(valid_smis)]
    unique_rate = len(unique_smis) / len(sample_smis)

    print(f'valid {valid_rate*100:.3f}%  |  unique {unique_rate*100:.3f}%')

    from rdkit.Chem.Draw import IPythonConsole, MolsToGridImage

    MAX_GRID = 16
    img = MolsToGridImage(
        [m(smi) for smi in unique_smis[:MAX_GRID]],
        molsPerRow=int(math.sqrt(MAX_GRID)),
        subImgSize=(250, 150),  returnPNG=False)
    display(img)
    if return_valid_rate: return valid_rate
    else:  # could be used to visualize latent space
        return pd.DataFrame(unique_smis, columns=['smiles'])

In [ ]:

def recon_test(model):
    fed = 'Cc1cnc(nc1Nc1cccc(c1)S(=O)(=O)NC(C)(C)C)Nc1ccc(cc1)OCCN1CCCC1'
    test_smiles = [fed]
    te = [string2tensor(string, device=device) for string in test_smiles]

    y = model.reconstruct(te)
    y_mu = model.reconstruct_mu(te)

    temp = 1
    y = F.softmax(y / temp, dim=-1)
    y = torch.argmax(y, dim=-1)
    l = y.cpu().detach().tolist()

    y_mu = torch.argmax(F.softmax(y_mu/temp, dim=-1), dim=-1)
    l_mu = y_mu.cpu().detach().tolist()
    del te
    del y
    del y_mu

    # print('true: ', model.string2tensor(fed))
    # print('pred with noise:', l)
    # print('pred with mu:   ', l_mu)
    smi_recon = [ids2string(ids, rem_bos=True, rem_eos=True) for ids in l]
    smi_recon_mu = [ids2string(ids, rem_bos=True, rem_eos=True) for ids in l_mu]
    for i, j in zip(smi_recon, smi_recon_mu):
        # print(i)
        print('fed smi    :', fed)
        print('pred_z  smi:', i)
        print('pred_mu smi:', j)
        try:
            mol = m(i)
            if mol != None: display(mol)
            else: print('pred_z  mol is none, cannot recon')
        except: print('invalid pred smi from z, cannot recon')
        try:
            mol = m(j)
            if mol != None: display(mol)
            else: print('pred_mu mol is none, cannot recon')
        except: print('invalid pred smi from mu, cannot recon')


def _train_epoch(model, epoch, train_loader, loader, kl_weight, optimizer=None):
    if optimizer is None:
        model.eval()
    else:
        model.train()

    kl_loss_values = CircularBuffer(n_last)
    recon_loss_values = CircularBuffer(n_last)
    loss_values = CircularBuffer(n_last)
    classifier_loss_values = CircularBuffer(n_last)

    # mol_num = 20000
    # if mol_num > len(test): loader = gen_loader(test)
    # else: loader = gen_loader(test.sample(n=mol_num, random_state=1))

    for i, batch in tqdm(enumerate(train_loader), total=len(train_loader)):
        if len(batch) == 2: x, label = batch
        else: x, label = batch, None
        # input_batch = tuple(data.to(device) for data in x)

    #forward
        kl_loss, recon_loss, classifier_loss = model(x, label)
        del x
        del label
        del batch
        loss = kl_weight * kl_loss + recon_loss + \
                pred_loss_percent * classifier_loss

    #backward
        if optimizer is not None:
            optimizer.zero_grad()
            loss.backward()
            clip_grad_norm_(get_optim_params(model), clip_grad)
            optimizer.step()

        kl_loss_values.add(kl_loss.item())
        recon_loss_values.add(recon_loss.item())
        classifier_loss_values.add(classifier_loss.item())
        loss_values.add(loss.item())
        lr = (optimizer.param_groups[0]['lr'] if optimizer is not None else None)

    #update train_loader
        kl_loss_value = kl_loss_values.mean()
        recon_loss_value = recon_loss_values.mean()
        classifier_loss_value = classifier_loss_values.mean()
        loss_value = loss_values.mean()
        postfix = [f'loss={loss_value:.5f}',f'(kl={kl_loss_value:.5f}',
                   f'recon={recon_loss_value:.5f})',
                   f'classify={classifier_loss_value:.5f}',
                   f'klw={kl_weight:.5f} lr={lr:.5f}']
        # print(i)
        if i % (int(len(train_loader)/10)) == 0 and i!= 0:
            print(f'Epoch {epoch}: {i}/{len(train_loader)} |', postfix)
            # if loader != None:
            #     fig_name = int(i/(len(train_loader)/10))
            #     visual_latent_space(loader, model, f"{epoch}_{fig_name}", display_fig=False)

    postfix = {'epoch': epoch,'kl_weight': kl_weight,'lr': lr,
               'kl_loss': kl_loss_value,'recon_loss': recon_loss_value,
               'loss': loss_value,'mode': 'Eval' \
               if optimizer is None else 'Train'}
    sample_smis = sample.take_samples(model, n_batch)
    valid_rate = visual_mols(sample_smis)

    recon_test(model)
    del loader
    model.train()
    # print(postfix)
    return postfix, valid_rate

def _train(model, train_loader, val_loader=None, logger=None):
    optimizer = optim.Adam(get_optim_params(model),lr= lr_start)

    lr_annealer = CosineAnnealingLRWithRestart(optimizer)
    valid_rate_best = 0
    model.zero_grad()
    for epoch in range(n_epoch):

        kl_annealer = KLAnnealer(n_epoch)
        kl_weight = kl_annealer(epoch)

        postfix, valid_rate = _train_epoch(model, epoch, train_loader, None, kl_weight, optimizer)

        # visual_latent_space(loader, model, f"{epoch}")
        # del loader

        # visual_latent_space(loader, model, f"{epoch}")
        if valid_rate > valid_rate_best:
            valid_rate_best = valid_rate
            if valid_rate > 0.5:
                print('#'*20)
                print('save model ckpt.pt')
                torch.save(model.state_dict(), 'ckpt.pt')

                # print("visual latent space")
                # mol_num = 20000
                # loader = gen_loader(test.sample(n=mol_num, random_state=1))
                # visual_latent_space(loader, model, f"{epoch}")
                # del loader

        elif valid_rate < 0.75 * valid_rate_best:
            print('valid rate drop, early stop')
            break
        lr_annealer.step()

def fit(model, train_data, val_data=None, classifier=None):
    logger = Logger() if False is not None else None
    if classifier == None:

        train_loader = get_dataloader(model, train_data, shuffle=True)

        val_loader = None if val_data is None else get_dataloader(model, val_data, shuffle=False)
    else:
        train_loader = train_data
        val_loader = val_data

    _train(model, train_loader, val_loader, logger)
    return model



In [ ]:
model = VAE(vocab,vector).to(device)

loader_jak = DataLoader(jak_dataset(train, property_names, device=device,
                                    mask=MASK),batch_size=n_batch, shuffle=True,
                        num_workers=n_workers, collate_fn=my_collate)
fit(model, loader_jak, val_data=None, classifier=True)


In [ ]:
from transformers import BertModel, BertTokenizer
import re
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )
model = BertModel.from_pretrained("Rostlab/prot_bert")
sequence_Example = "A E T C Z A O"

# sequence_Example = 'AETCZAO'
sequence_Example = re.sub(r"[UZOB]", "X", sequence_Example)
encoded_input = tokenizer(sequence_Example, return_tensors='pt')
output = model(**encoded_input)

In [ ]:
! pip install transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 124.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.4 MB/s eta 0:00:00


In [ ]:
output.items()

odict_items([('last_hidden_state', tensor([[[ 0.0454,  0.1140, -0.0117,  ..., -0.0875, -0.1143,  0.0204],
         [ 0.0923,  0.1391, -0.0524,  ..., -0.1395, -0.0428,  0.0743],
         [ 0.1151,  0.0200, -0.0863,  ..., -0.0095, -0.1873,  0.1317],
         ...,
         [ 0.1079,  0.0977, -0.0583,  ..., -0.1277, -0.0649,  0.1289],
         [ 0.0546,  0.0364, -0.0782,  ..., -0.0302, -0.0602,  0.0890],
         [ 0.0515,  0.0571, -0.0693,  ..., -0.0394, -0.0663,  0.0977]]],
       grad_fn=<NativeLayerNormBackward0>)), ('pooler_output', tensor([[-0.2487,  0.2626, -0.2367,  ...,  0.2503,  0.2339, -0.2556]],
       grad_fn=<TanhBackward0>))])

In [ ]:
output['pooler_output'].shape

torch.Size([1, 1024])

In [ ]:
output.items()

odict_items([('last_hidden_state', tensor([[[-0.1096,  0.0474, -0.0857,  ..., -0.0035, -0.0569,  0.0918],
         [-0.0628,  0.0624, -0.0855,  ...,  0.0178, -0.0978,  0.0628],
         [-0.1252,  0.0657, -0.1055,  ..., -0.0381,  0.0185,  0.0819]]],
       grad_fn=<NativeLayerNormBackward0>)), ('pooler_output', tensor([[-0.1668,  0.1769, -0.1581,  ...,  0.1679,  0.1562, -0.1718]],
       grad_fn=<TanhBackward0>))])

In [ ]:
output.items()

odict_items([('last_hidden_state', tensor([[[ 0.0454,  0.1140, -0.0117,  ..., -0.0875, -0.1143,  0.0204],
         [ 0.0923,  0.1391, -0.0524,  ..., -0.1395, -0.0428,  0.0743],
         [ 0.1151,  0.0200, -0.0863,  ..., -0.0095, -0.1873,  0.1317],
         ...,
         [ 0.1079,  0.0977, -0.0583,  ..., -0.1277, -0.0649,  0.1289],
         [ 0.0546,  0.0364, -0.0782,  ..., -0.0302, -0.0602,  0.0890],
         [ 0.0515,  0.0571, -0.0693,  ..., -0.0394, -0.0663,  0.0977]]],
       grad_fn=<NativeLayerNormBackward0>)), ('pooler_output', tensor([[-0.2487,  0.2626, -0.2367,  ...,  0.2503,  0.2339, -0.2556]],
       grad_fn=<TanhBackward0>))])

In [ ]:
! pip install PyTDC --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 58.9 MB/s eta 0:00:00


In [ ]:
from tdc.multi_pred import DTI
data = DTI(name = 'BindingDB_Kd')
# data.harmonize_affinities(mode = 'max_affinity')
data.harmonize_affinities(mode = 'mean')

Downloading...
100%|██████████| 54.4M/54.4M [00:06<00:00, 7.83MiB/s]
Loading...
Done!
The original data has been updated!


,Drug_ID,Drug,Target_ID,Target,Y
0,51.0,O=C(O)CCC(=O)C(=O)O,Q9GZT9,MANDSGGPGGPSPSERDRQYCELCGKMENLLRCSRCRSSFYCCKEH...,1450.0
1,187.0,CC(=O)OCC[N+](C)(C)C,P11229,MNTSAPPAVSPNITVLAPGKGPWQVAFIGITTGLLSLATVTGNLLV...,17378.0
2,187.0,CC(=O)OCC[N+](C)(C)C,P58154,MRRNIFCLACLWIVQACLSLDRADILYNIRQTSRPDVIPTQRDRPV...,2154.0
3,237.0,CCN(CC)CCCC(C)Nc1c2ccc(Cl)cc2nc2ccc(OC)cc12,P02752,MLRFAITLFAVITSSTCQQYGCLEGDTHKANPSPEPNMHECTLYSE...,264.0
4,242.0,O=C([O-])c1ccccc1,P14920,MRVVVIGAGVIGLSTALCIHERYHSVLQPLDIKVYADRFTPLTTTD...,4500.0
...,...,...,...,...,...
42231,138805907.0,COc1nc2ccc([C@@](O)(c3ccc(C(F)(F)F)nc3)c3cncn3...,P51449,MDRAPQRQHRASRELLAAKKTHTSQIEVIPCKICGDKSSGIHYGVI...,3.1
42232,138805908.0,COc1nc2ccc([C@](O)(c3ccnc(C(F)(F)F)c3)c3cncn3C...,P51449,MDRAPQRQHRASRELLAAKKTHTSQIEVIPCKICGDKSSGIHYGVI...,9300.0
42233,138805909.0,COc1nc2ccc([C@@](O)(c3ccnc(C(F)(F)F)c3)c3cncn3...,P51449,MDRAPQRQHRASRELLAAKKTHTSQIEVIPCKICGDKSSGIHYGVI...,3.5
42234,138805910.0,COc1nc2ccc([C@](O)(c3ccc(Cl)cc3)c3cncn3C)cc2c(...,P51449,MDRAPQRQHRASRELLAAKKTHTSQIEVIPCKICGDKSSGIHYGVI...,850.0


In [ ]:
split['train']

,Drug_ID,Drug,Target_ID,Target,Y
0,51.0,O=C(O)CCC(=O)C(=O)O,Q9GZT9,MANDSGGPGGPSPSERDRQYCELCGKMENLLRCSRCRSSFYCCKEH...,1450.0
1,187.0,CC(=O)OCC[N+](C)(C)C,P58154,MRRNIFCLACLWIVQACLSLDRADILYNIRQTSRPDVIPTQRDRPV...,2154.0
2,264.0,CCCC(=O)O,Q9Z429,MQQFTIRTRLLMLVGAMFIGFITIELMGFSALQRGVASLNTVYLDR...,92000.0
3,311.0,O=C(O)CC(O)(CC(=O)O)C(=O)O,P15474,MPRSLANAPIMILNGPNLNLLGQRQPEIYGSDTLADVEALCVKAAA...,7300.0
4,525.0,O=C(O)CC(O)C(=O)O,Q9Z429,MQQFTIRTRLLMLVGAMFIGFITIELMGFSALQRGVASLNTVYLDR...,8500.0
...,...,...,...,...,...
29560,138805907.0,COc1nc2ccc([C@@](O)(c3ccc(C(F)(F)F)nc3)c3cncn3...,P51449,MDRAPQRQHRASRELLAAKKTHTSQIEVIPCKICGDKSSGIHYGVI...,3.1
29561,138805908.0,COc1nc2ccc([C@](O)(c3ccnc(C(F)(F)F)c3)c3cncn3C...,P51449,MDRAPQRQHRASRELLAAKKTHTSQIEVIPCKICGDKSSGIHYGVI...,9300.0
29562,138805909.0,COc1nc2ccc([C@@](O)(c3ccnc(C(F)(F)F)c3)c3cncn3...,P51449,MDRAPQRQHRASRELLAAKKTHTSQIEVIPCKICGDKSSGIHYGVI...,3.5
29563,138805910.0,COc1nc2ccc([C@](O)(c3ccc(Cl)cc3)c3cncn3C)cc2c(...,P51449,MDRAPQRQHRASRELLAAKKTHTSQIEVIPCKICGDKSSGIHYGVI...,850.0


In [ ]:
# from tdc.chem_utils import MolConvert
# converter = MolConvert(src = 'SMILES', dst = 'Graph3D')

In [ ]:
data.convert_to_log(form='binding')

To log space...


In [ ]:
split = data.get_split()

split['train']

,Drug_ID,Drug,Target_ID,Target,Y
0,51.0,O=C(O)CCC(=O)C(=O)O,Q9GZT9,MANDSGGPGGPSPSERDRQYCELCGKMENLLRCSRCRSSFYCCKEH...,5.838602
1,187.0,CC(=O)OCC[N+](C)(C)C,P58154,MRRNIFCLACLWIVQACLSLDRADILYNIRQTSRPDVIPTQRDRPV...,5.666734
2,264.0,CCCC(=O)O,Q9Z429,MQQFTIRTRLLMLVGAMFIGFITIELMGFSALQRGVASLNTVYLDR...,4.036212
3,311.0,O=C(O)CC(O)(CC(=O)O)C(=O)O,P15474,MPRSLANAPIMILNGPNLNLLGQRQPEIYGSDTLADVEALCVKAAA...,5.136671
4,525.0,O=C(O)CC(O)C(=O)O,Q9Z429,MQQFTIRTRLLMLVGAMFIGFITIELMGFSALQRGVASLNTVYLDR...,5.070576
...,...,...,...,...,...
29560,138805907.0,COc1nc2ccc([C@@](O)(c3ccc(C(F)(F)F)nc3)c3cncn3...,P51449,MDRAPQRQHRASRELLAAKKTHTSQIEVIPCKICGDKSSGIHYGVI...,8.494850
29561,138805908.0,COc1nc2ccc([C@](O)(c3ccnc(C(F)(F)F)c3)c3cncn3C...,P51449,MDRAPQRQHRASRELLAAKKTHTSQIEVIPCKICGDKSSGIHYGVI...,5.031512
29562,138805909.0,COc1nc2ccc([C@@](O)(c3ccnc(C(F)(F)F)c3)c3cncn3...,P51449,MDRAPQRQHRASRELLAAKKTHTSQIEVIPCKICGDKSSGIHYGVI...,8.443697
29563,138805910.0,COc1nc2ccc([C@](O)(c3ccc(Cl)cc3)c3cncn3C)cc2c(...,P51449,MDRAPQRQHRASRELLAAKKTHTSQIEVIPCKICGDKSSGIHYGVI...,6.070530
